In [ ]:
IS_COLAB = True

if IS_COLAB:
  from google.colab import drive

  drive.mount('/gdrive')
  pj_home = '/gdrive/My Drive/colab/3-min-pytorch/ch7'

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


# 7. 순차적인 데이터를 처리하는 RNN

## 7.0. 요약

### 7.0.1. 지난장

지난 장에서는 일반적인 지도학습의 방법과 달리 label $y$가 주어지지 않는 상태에서 학습된 결과 $\hat{x}$이 입력값인 $x$와의 오차를 활용하여 학습하는 비지도방식을 활용하는 오토인코더에 대해서 알아보았다.

오토인코더는 크게 encoder와 decoder부분으로 나뉘며 각각의 예제를 통해 encoder로 추출한 잠재요인이 얼마나 잘 대표하여 데이터를 표현하는지 시각적으로도 살펴보았고, decoder를 통해 얼마나 결과가 복원되는지에 대해도 살펴보았다.

### 7.0.2. 이번장

이번 장에서는 앞서 살펴보았던 신경망들에서 학습할 수록 이전의 정보들에 대해서 망각하는 문제를 보완한 순환신경망(RNN, Recurrent Neural Network)에 대해서 알아보자.

## 7.1. RNN 개요

앞서 학습한 신경망들에서 망각하고 있는 문제는 연달아 있는 데이터의 순서와 상호작용을 인식하여 전체상황을 이해하는 능력이다. 다시 말해, 주어지는 input에 대해 그 시점에 대한 정적인 데이터만을 학습하고 결과를 return한다.

그러나 현실에서 주어지는 데이터는 대부분 순차적이다. 그러므로 RNN을 통해 데이터가 순서적으로 나열된 순차적 데이터 또는 시계열 데이터의 정보를 받아 전체내용을 학습해보려고 한다.

RNN의 출력은 순차적인 데이터의 흐름을 모두 내포한다. 시계열 데이터의 정보를 하나씩 입력받을 떄 마다 지금까지 입력된 벡터들을 종합한 은닉벡터를 만들어낸다.

### RNN의 기본 구조
<img src="img/RNN_single.png"/>

기본적인 RNN 모형은 위와 같다. 아래의 $x_{0,1,2,3,4}$에 해당하는 것들은 각 시점의 순차적인 데이터의 input이며 $A_{0,1,2,3,4}$는 각 시점의 데이터 input과 그 전 시점의 은닉벡터 $A_{n-1}$의 입력을 합한 은닉벡터이다. 추후 가장 마지막의 은닉벡터는 순차적으로 입력된 데이터가 모두 반영되고 압축된 벡터라고 할 수 있다.

RNN 계열의 신경망들은 대표적인 시계열 데이터인 텍스트및 자연어를 처리하고 학습하는데 주로 사용되며, LSTM, GRU등의 응용 RNN을 통해 언어 모델링, 텍스트 감정 분석, 기계 번역등의 분야에서 활용되고 있다.

### RNN의 다양한 구조
<img src="img/RNN_structure.png"/>

1. 일대일: 일반적으로 살펴보았던 신경망이나 CNN과 동일

2. 일대다: 이미지를 보고 이미지 안의 상황을 글로 설명하는 문제

3. 다대일: 감정 분석과 같은 순차적인 데이터를 보고 값 하나를 내는 경우

4. 다대다 (1): 챗봇과 기계 번역 과 같이 순차적인 데이터를 보고 순차적인 데이터를 출력하는 문제

5. 다대다 (2): 비디오 분류와 같이매 프레임을 레이블링할 때 사용되는 구조

## 7.2. 영화 리뷰 감정 분석

영화 리뷰 감정 분석을 위해 IMDB의 영화 데이터셋을 활용할 것이며, 각 영화의 리뷰 텍스트를 입력값을 활용하여 전체를 압축하고 압축된 리뷰가 긍정적인지 부정적인지 판단하는 간단한 분류 모델을 만들 것이다.

### ※ 토크나이징과 워드 임베딩
IMDB 데이터셋은 학습해야 하는 데이터가 텍스트로 구행되어 있다. 이를 학습하기 위해서는 전처리 과정을 통해 데이터를 숫자로 나타내야 한다. 우선 텍스트 데이터를 특정한 작은 단위로 나누는 토크나이징을 수행한다. 그리고 각각의 토큰을 벡터로 변환한다. 이런 두가지의 전반적인 절차를 일컬어 워드 임베딩이라고 한다.

### 7.2.1. 자연어 전처리

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext import data ,datasets

In [ ]:
BATCH_SIZE=64
lr=0.001
EPOCHS=10
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

데이터로 주어지는 입력값인 텍스트와 라벨을 어떻게 구성하는지 정의한다.

참고: <a href="https://simonjisu.github.io/nlp/2018/07/18/torchtext.html">Torchtext Tutorial</a>

In [ ]:
TEXT = data.Field(sequential=True, batch_first=True, lower=True)
LABEL = data.Field(sequential=False, batch_first=True)

In [ ]:
train_dataset, test_dataset= datasets.IMDB.splits(TEXT,LABEL)

In [ ]:
TEXT.build_vocab(train_dataset,min_freq=5)
LABEL.build_vocab(train_dataset)

In [ ]:
train_dataset, valid_dataset = train_dataset.split(split_ratio=0.8)
train_iter, val_iter, test_iter = data.BucketIterator.splits(
                                            (train_dataset, valid_dataset, test_dataset),
                                            batch_size=BATCH_SIZE,
                                            shuffle=True, repeat=False)

In [ ]:
vocab_size = len(TEXT.vocab)
n_classes = 2

In [ ]:
print(f"[훈련셋]: {len(train_dataset)}; [검증셋]: {len(valid_dataset)}; [테스트셋]: {len(test_dataset)}; [단어수]: {vocab_size}; [클래스]: {n_classes}")

[훈련셋]: 20000; [검증셋]: 5000; [테스트셋]: 25000; [단어수]: 46159; [클래스]: 2


### 7.2.2. RNN 모델 구현

In [ ]:
'''
n_layers: 은닉벡터들의 층 - 복잡한 모델이 아닌 경우를 제외하면 2 이하로 설정
n_vocab: 전체 데이터 셋의 단어를 사전화했을 때 등재된 단어의 수
embed_dim: 임베딩 된 단어 텐서가 지니는 차원 값
'''

class BasicGRU(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
        super(BasicGRU,self).__init__()
        print("Building GRU Model...")
        
        self.n_layers = n_layers
        
        self.embed = nn.Embedding(n_vocab, embed_dim)
        
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(dropout_p)
        
        self.gru = nn.GRU(embed_dim, self.hidden_dim, num_layers=n_layers, batch_first=True)
        
        self.out = nn.Linear(self.hidden_dim, n_classes)
        
    def forward(self, x):
        x = self.embed(x)
        
        # RNN 계열 신경망을 위한 첫번째 은닉벡터 H0 정의
        h_0 = self._init_state(batch_size=x.size(0))
        
        x, _ = self.gru(x, h_0)
        h_t = x[:,-1,:]
        self.dropout(h_t)
        
        logit = self.out(h_t)
        return logit
        
    '''
    nn.Module의 parameter() 함수는 신경망의 가중치 정보들을 반복자형태로 보관하고 있음.
    따라서, next를 활용하여 GRU 모듈의 첫번째 은닉벡터에 대한 가중치를 뽑고 이를 new 함수로
    기존의 가중치와 동일한 형태로 추출하여 0으로 가중치를 변경하는 것.
    
    따라서, 첫번째 은닉 벡터 H0는 모든 특성 값이 0인 벡터로 설정된다.
    '''    
    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

#### ※ GRU?

RNN 모델을 구현한다고 언급했는데 왜 GRU 패키지를 사용하는가? 여기에는 문제점이 있기 때문이다. 기본적인 RNN은 입력이 길어지면 학습 도중 기울기가 너무 작아지거나 커져 앞부분에 대한 정보를 정확히 담기 어렵다. 이를 RNN 학습 도중 기울기가 커지는 현상을 Gradient Explosion이라고 하고, 반대로 작아지는 현상을 Gradient Descent라고 한다.이러한 문제점을 보완한 모델으 GRU(gated Recurrent Unit)이며, 시계열 데이터 속 벡터 사이의 정보 전달량을 조절함으로써 기울기를 적정하게 유지하고 문장 앞부분의 정보가 끝까지 도달할 수 있도록 도와준다.

* 업데이트 게이트: 이전 은닉벡터가 지닌 정보를 새로운 은닉벡터가 얼마나 유지할 지 정함

* 리셋 게이트: 새로운 입력이 이전 은닉 벡터와 어떻게 조합하는 지 결정

In [ ]:
def train(model, optimizer, train_iter):
    model.train()
    
    for b, batch in enumerate(train_iter):
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1)
        
        optimizer.zero_grad()
        logit = model(x)
        
        loss = F.cross_entropy(logit, y)
        loss.backward()
        optimizer.step()
        
def evaluate(model, val_iter):
    model.eval()
    corrects, total_loss = 0, 0
    for batch in val_iter:
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1)
        logit = model(x)
        loss = F.cross_entropy(logit, y, reduction='sum')
        total_loss += loss.item()
        corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()
    size = len(val_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = 100.0 * corrects / size
    return avg_loss, avg_accuracy

In [ ]:
model = BasicGRU(n_layers=1, hidden_dim=256, n_vocab=vocab_size, embed_dim=128, n_classes=n_classes, dropout_p=0.5).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

Building GRU Model...


In [ ]:
best_val_loss = None
for e in range(1, EPOCHS+1):
    train(model,optimizer,train_iter)
    val_loss, val_acc = evaluate(model, val_iter)
    print(f"EPOCH: {e}; Valid Loss: {val_loss}; Valid Accuracy: {val_acc}")
    
    if not best_val_loss or val_loss < best_val_loss:
        if not os.path.isdir(pj_home+"/snapshot"):
            os.makedirs(pj_home+"/snapshot")
        torch.save(model.state_dict(), pj_home+'/snapshot/txtclassfication.pt')
        best_val_loss = val_loss

EPOCH: 1; Valid Loss: 0.6955183844566345; Valid Accuracy: 48.34000015258789
EPOCH: 2; Valid Loss: 0.6978997091293335; Valid Accuracy: 50.81999969482422
EPOCH: 3; Valid Loss: 0.6943544293403625; Valid Accuracy: 51.21999740600586
EPOCH: 4; Valid Loss: 0.6838338097572326; Valid Accuracy: 57.459999084472656
EPOCH: 5; Valid Loss: 0.659149230670929; Valid Accuracy: 60.79999923706055
EPOCH: 6; Valid Loss: 0.3535765057086945; Valid Accuracy: 86.0999984741211
EPOCH: 7; Valid Loss: 0.30951704885959624; Valid Accuracy: 87.31999969482422
EPOCH: 8; Valid Loss: 0.4356783332824707; Valid Accuracy: 82.9000015258789
EPOCH: 9; Valid Loss: 0.3569665704727173; Valid Accuracy: 86.95999908447266
EPOCH: 10; Valid Loss: 0.532422824382782; Valid Accuracy: 83.5199966430664


In [ ]:
model.load_state_dict(torch.load(pj_home+'/snapshot/txtclassfication.pt'))
test_loss, test_acc = evaluate(model,test_iter)
print(f'Test Loss: {test_loss}; Test Accuracy: {test_acc}')

Test Loss: 0.32830322525024414; Test Accuracy: 86.33599853515625


## 7.3. Seq2Seq 기계번역

Seq2Seq 모델은 시퀀스를 입력받아 또 다른 시퀀스를 출력한다. 다시 말해 하나의 문장을 다른 문장으로 번역해주는 모델을 말한다.

Seq2Seq는 각자 다른 역할을 하는 두 개의 RNN을 이어붙인 모델이다.
1. 외국어의 문장을 읽고 의미를 이해하는 RNN [인코더]
2. 외국어 문장의 의미를 생각하면서 한국어 단어를 문맥에 맞게 적어가는 RNN [디코더]

두개의 RNN은 위와 같이 오토인코더와 같이 인코더와 디코더로 구성됨을 알 수 있다.

### 7.3.2. 인코더

인코더의 구성은 앞선 기본적인 RNN의 구조에서 살펴본 것 처럼, 하나의 문장을 임의의 작은 단위로 나누어 input으로 구성한다. 각각의 input은 은닉층을 거쳐 은닉벡터를 구성하고 해당 은닉벡터는 이전 은닉벡터와 입력값을 반영하여 이전의 정보들을 순차적으로 학습하게 된다.

#### ※ Encoder Btw autoencoder and seq2seq

오토인코더의 인코더는 하나의 정적인 데이터에 대해 이를 잘 대표할 수 있는 몇개의 잠재요인으로 정보들을 추려냄으로써 차원수를 줄인다. 

Seq2Seq의 인코더는 하나의 동적인 데이터에 대해 정적인 문맥 단위로 나누고 은닉벡터에 순차적으로 넣음으로써 결국에는 하나의 정적인 데이터로 나타내는 것이 정보를 압축하는 것으로 볼 수 있다.

## 7.3.3. 디코더

디코더 또한 하나의 RNN 모델이며, 인코더에서 원문 문맥 벡터를 이어받아 번역문 속의 토큰을 차례대로 예상하고 생성한다.

### 7.3.4. Seq2Seq 모델 구현하기

In [1]:
import torch
import torch.nn as nn
import random
import matplotlib.pyplot as plt

In [2]:
vocab_size = 256

In [7]:
x_ = list(map(ord,"hello"))
y_ = list(map(ord,"hola"))
x = torch.LongTensor(x_)
y = torch.LongTensor(y_)

#### Teacher Forcing (티처포싱)

디코더 구조에서 임의의 단계의 입력을 통해 얻은 출력값이 다음 디코더 신경망의 입력값으로 들어가는 것이 옳지만, 학습 수준이 낮은  신경망의 경우에는 잘 못된 예측 토큰이 입력으로 사용될 확률이 높다. 이로 인해 학습이 더뎌질 수 있다.

이를 방지하기 위해서 디코더 학습 시 실제 번역문의 토큰을 디코더의 전 출력값 대신 입력으로 사용하여 학습을 가속하는 방법이다. 번역문의 i번째 토큰에 해당하는 값 targets[i]를 디코더의 입력값으로 설정한다.

In [26]:
class Seq2Seq(nn.Module):
  def __init__(self, vocab_size, hidden_size):
    super(Seq2Seq,self).__init__()
    self.n_layers = 1
    self.hidden_size = hidden_size

    self.embedding = nn.Embedding(vocab_size, hidden_size)
    self.encoder = nn.GRU(hidden_size, hidden_size)
    self.decoder = nn.GRU(hidden_size, hidden_size)

    self.project = nn.Linear(hidden_size, vocab_size)

  def forward(self, inputs , targets):
    initial_state = self._init_state()
    embedding = self.embedding(inputs).unsqueeze(1)

    encoder_output, encoder_state = self.encoder(embedding, initial_state)
    decoder_state = encoder_state
    decoder_input = torch.LongTensor([0])

    outputs = []
    for i in range(targets.size()[0]):
      decoder_input = self.embedding(decoder_input).unsqueeze(1)
      decoder_output, decoder_state = self.decoder(decoder_input, decoder_state)
      projection = self.project(decoder_output)
      outputs.append(projection)
      # 티처포싱 (Teacher Forcing)
      decoder_input = torch.LongTensor([targets[i]])
    
    outputs = torch.stack(outputs).squeeze()
    return outputs

  def _init_state(self, batch_size=1):
      weight = next(self.parameters()).data
      return weight.new(self.n_layers, batch_size, self.hidden_size).zero_()

In [27]:
seq2seq = Seq2Seq(vocab_size, 16)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(seq2seq.parameters(), lr=0.001)

log = []
for i in range(1000):
  prediction = seq2seq(x,y)
  loss = criterion(prediction, y)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  loss_val = loss.data
  log.append(loss_val)
  
  if i % 100 == 0:
    print("\n 반복: %d 오차: %s"%(i,loss_val.item()))
    _, top1 = prediction.data.topk(1,1)
    print([chr(c) for c in top1.squeeze().numpy().tolist()])


 반복: 0 오차: 5.5043044090271
['\\', '\x83', 'á', 'á']

 반복: 100 오차: 1.9007099866867065
['h', 'a', 'a', 'a']

 반복: 200 오차: 0.6240732073783875
['h', 'o', 'l', 'a']

 반복: 300 오차: 0.31990742683410645
['h', 'o', 'l', 'a']

 반복: 400 오차: 0.1955108940601349
['h', 'o', 'l', 'a']

 반복: 500 오차: 0.13071636855602264
['h', 'o', 'l', 'a']

 반복: 600 오차: 0.09475205838680267
['h', 'o', 'l', 'a']

 반복: 700 오차: 0.07284270972013474
['h', 'o', 'l', 'a']

 반복: 800 오차: 0.05827301740646362
['h', 'o', 'l', 'a']

 반복: 900 오차: 0.04793937876820564
['h', 'o', 'l', 'a']
